In [37]:
import pandas as pd
import gsw
from pathlib import Path

In [38]:
lat = None; lon = None
filename = Path(r'd:\WorkData\_source\Первичные данные SBE-19.xls')
my_cols_names = {
"Dat":     'Дата, мм/дд/гггг',
"Tim":     'Время, чч:мм:сс',
"Pres":    'Давление, дБар',
"Depth":   'Глубина, м',
"Temp90":  'Температура,°C',
"Temp_pot":'Потенциальная температура,°C',
"C":       'Электропроводность, См/м',
"Sal":     'Соленость, епс',
"O2ppm":   'Содержание кислорода, мл/л',
"O2":      'Насыщение кислородом, %',
"Turb":    'Мутность, емф',
"SoundV":  'Скорость звука (Chen-Millero), м/с',
"st":      'Станция',
"Lat":     'Широта',
"Lon":     'Долгота'
}

In [39]:
xls = pd.read_excel(filename, index_col=0, parse_dates= {'Time': [0,1]}, names= my_cols_names.keys(), sheet_name=None)
print(f'writing sheets: {xls.keys()}')

In [56]:
with pd.ExcelWriter(filename.with_name(filename.stem + '_mod.xlsx')) as writer:
    for sheet_name, ctd in xls.items():
        ctd['SA'] = gsw.SA_from_SP(ctd['Sal'], ctd['Pres'], lat=ctd.get('Lat', lat), lon=ctd.get('Lon', lon))
        ctd['CT'] = gsw.CT_from_t(ctd['SA'], ctd['Temp90'], ctd['Pres'])
        ctd['sigma0'] = gsw.sigma0(ctd['SA'], ctd['CT'])
        ctd.rename(columns={my_cols_names, *{'Time': 'Дата, Время'}}).to_excel(writer, sheet_name=sheet_name)